In [37]:
import os
import pandas as pd

import py2neo
from py2neo import Graph,Node,Relationship

import IPython

print ('py2neo version', py2neo.__version__)

py2neo version 2021.2.3


In [38]:
db_url = 'neo4j@bolt://neo4j@localhost:7687'
db_pass = 'tweets'

graph = Graph(db_url, password = db_pass)

In [39]:
#consulta de prueba para ver que todo funcione

nro_nodos_user="MATCH(u:User) RETURN Count(u)"
nro_nodos_tweet="MATCH(t:Tweet) RETURN Count(t)"

#evaluamos las consultas
result_users=graph.evaluate(nro_nodos_user)
result_tweets=graph.evaluate(nro_nodos_tweet)

#imprimo resultados
print(f'En la BD hay {result_users} nodos User y {result_tweets} nodos Tweet')

En la BD hay 55860 nodos User y 405369 nodos Tweet


In [41]:
# query for dropping a projection from the catalog
query = (
    "CALL gds.graph.drop('tweets') "
)

result = graph.run(query).to_data_frame()

result

,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,tweets,neo4j,,-1,474112,1098318,{'relationshipProjection': {'QUOTE': {'orienta...,0.000005,2022-06-13T18:32:47.873189000-03:00,2022-06-13T18:32:50.980297999-03:00,"{'relationships': {'QUOTE': {}, 'TAG': {}, 'ME..."


In [48]:
# Create a projection of the graph and store it on the catalog
# We use the user projection described in https://towardsdatascience.com/using-neo4j-graph-database-to-analyze-twitter-data-6e3d38042af1
user_projecion = 'MATCH (u:User) return id(u) as id'
relationships_projection = (
    "MATCH (u1:User)-[:TWEETED]->(t1:Tweet)<-[:QUOTE|REPLY]-(t2:Tweet)<-[:TWEETED]-(u2:User) "
    "RETURN id(u2) as source, id(u1) as target "
    "UNION "
    "MATCH (u1:User)<-[:MENTIONS]-(t1:Tweet)<-[:QUOTE|REPLY]-(t2:Tweet)<-[:TWEETED]-(u2:User) "
    "RETURN id(u2) as source, id(u1) as target "
    "UNION "
    "MATCH (u1:USER)<-[:MENTIONS]-(t1:Tweet)<-[:TWEETED]-(u2:User) "
    "RETURN id(u2) as source, id(u1) as target "
)

nodos_tweet = (
    f"CALL gds.graph.project.cypher('tweets', '{user_projecion}', '{relationships_projection}') "
)

#evaluamos las consultas
tweets=graph.run(nodos_tweet).to_data_frame() 

tweets

,nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
0,MATCH (u:User) return id(u) as id,MATCH (u1:User)-[:TWEETED]->(t1:Tweet)<-[:QUOT...,tweets,55860,187877,3515


In [49]:
# Retrieve all graphs on the catalog to see if it was stored correctly

query = (
    "CALL gds.graph.list() "
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 57, 'min': 0, 'max': 961, 'mean': 3.36...",tweets,neo4j,8717 KiB,8926907,55860,187877,{'relationshipQuery': 'MATCH (u1:User)-[:TWEET...,0.00006,2022-06-21T19:13:03.148202000-03:00,2022-06-21T19:13:06.701191000-03:00,"{'relationships': {'__ALL__': {}}, 'nodes': {'..."


In [50]:
# Execute the PageRank centrality algorithm

query = (
    "CALL gds.pageRank.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Telemundo,TelemundoUY,281014,147473,204.974342
1,Subrayado,Subrayado,833617,200657,129.424268
2,EL PAÍS,elpaisuy,792004,417586,121.168402
3,leo sarro press,leosarro,51348,14180,114.979262
4,PEÑAROL | Basketball,BasketCAPuy,33843,3006,85.854922
5,Isabel_,Isabel66991411,5478,78777,67.521515
6,Inumet,MeteorologiaUy,146749,7438,57.485985
7,Nacional,Nacional,341359,29775,50.366663
8,MSP - Uruguay,MSPUruguay,164823,15172,37.949762
9,Montevideo Portal,portalmvd,571372,456315,37.355354


In [52]:
# Execute the ArticleRank centrality algorithm

query = (
    "CALL gds.articleRank.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,16.372308
1,Nacional,Nacional,341359,29775,14.412332
2,PEÑAROL,OficialCAP,434933,43753,11.890014
3,Telemundo,TelemundoUY,281014,147473,11.822032
4,Montevideo Portal,portalmvd,571372,456315,9.218680
5,Luis Lacalle Pou,LuisLacallePou,416367,27270,8.952443
6,EL PAÍS,elpaisuy,792004,417586,8.804899
7,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,8.525932
8,Subrayado,Subrayado,833617,200657,8.059107
9,leo sarro press,leosarro,51348,14180,7.796260


In [54]:
# Execute the Betweenness centrality algorithm

query = (
    "CALL gds.betweenness.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,3.662321e+07
1,☪️🅰️®️🛴🌎💰,boniatero75,2129,42533,2.682564e+07
2,gaby 💖💙❤️,sgabyo,2084,28238,2.297714e+07
3,𝕴𝖓𝖘𝖔𝖕𝖔𝖗𝖙𝖆𝖇𝖑𝖊𝖒𝖊𝖓𝖙𝖊 𝕭𝖔𝖑𝖘𝖔,KiwiNacional,1355,53954,1.675676e+07
4,LMP,Lelengo86,513,5257,1.669779e+07
5,Antonella Gordillo 🇺🇾🥕,AntoGordillo,5616,14396,1.615455e+07
6,Jorge Andrés,JorgeAndresBusi,2554,94890,1.609382e+07
7,👑🌻𝓐𝓷𝓽𝓸𝓷𝓮𝓵𝓵𝓪💎👑,Antonnella1891,3718,63855,1.352389e+07
8,Carmen rinaldi❤⚪💙,CarmenRinaldi4,5453,147724,1.332232e+07
9,Nati,NatiMArbiza,1643,53603,1.280733e+07


In [53]:
# Execute the Degree centrality algorithm

query = (
    "CALL gds.degree.stream('tweets') "
    "YIELD nodeId, score "
    "WITH gds.util.asNode(nodeId) AS node, score "
    "WHERE node.name IS NOT NULL "
    "RETURN node.name, node.username, node.followers, node.tweet_count, score "
    "ORDER BY score DESC, node.name ASC "
    "LIMIT 20"
)

#evaluamos las consultas
tweets=graph.run(query).to_data_frame() 

tweets

,node.name,node.username,node.followers,node.tweet_count,score
0,Isabel_,Isabel66991411,5478,78777,961.0
1,máximo j gutierrez z,bonnevilleminas,894,83471,912.0
2,Olga Alonso,OlgaAlonso9,1206,41509,788.0
3,gaby 💖💙❤️,sgabyo,2084,28238,773.0
4,Jorge Andrés,JorgeAndresBusi,2554,94890,747.0
5,Jenny Parada Martino,MartinoParada,1137,62402,662.0
6,Fernandomassa,Fernand47402543,2017,11601,581.0
7,Gabriela,zurdayo,3573,31947,566.0
8,𝕴𝖓𝖘𝖔𝖕𝖔𝖗𝖙𝖆𝖇𝖑𝖊𝖒𝖊𝖓𝖙𝖊 𝕭𝖔𝖑𝖘𝖔,KiwiNacional,1355,53954,532.0
9,Ángela Ruso🇺🇾🌎🌹🌹🥀,anyelinaruso,2740,15821,499.0
